In [128]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [129]:
df = pd.read_csv('shows.csv')

df

,id,feed_url,title,subtitle,description,summary,author,email,link,language,explicit,image,category,subcategory,created_at,last_build_date
0,0002da5f-49a2-31ba-b44c-cdeabdf113cb,http://www.cbc.ca/podcasting/includes/dispatch...,Dispatches from CBC Radio,NaN,CBC Radio's Dispatches host Rick MacInnes-Rae ...,CBC Radio's Dispatches host Rick MacInnes-Rae ...,CBC Radio,podcasting@cbc.ca,http://www.cbc.ca/podcasting,en-ca,0,http://www.cbc.ca/podcasting/images/promo-disp...,News & Politics,Public Radio,2016-11-27T03:15:54,2016-11-27T03:15:54
1,00044845-beb8-35f3-99e2-01dbcfb6eb63,http://everydayissaturday.com/feed/podcast/,Motivation | Inspiration| Success with Entrepr...,Motivation - Inspiration - Success - Mindset -...,Sam Crowley delivers million dollar motivation...,Sam Crowley delivers million dollar motivation...,Sam Crowley,sam@everydayissaturday.com,http://everydayissaturday.com,en-us,0,http://everydayissaturday.com/wp-content/uploa...,Business,Business,2016-11-26T19:11:59,2016-11-21T14:19:34
2,00048bfa-8363-3f10-ac8e-8f27a31fe11f,http://disability411.jinkle.com/disability411.xml,Disability411 Podcast,NaN,Disability411 - The Podcast for Disability Inf...,NaN,Beth Case,NaN,http://disability411.jinkle.com/,en-us,0,NaN,Government & Organizations,Business,2016-11-27T00:37:30,2016-11-27T00:37:30
3,0005ab14-ea88-3038-b82c-86366713e96a,http://stevenredant.podomatic.com/rss2.xml,Steven Redant's Podcast,"Energy for your work out, your travels and you...",En example of what happened on my dance floor ...,En example of what happened on my dance floor ...,Steven Redant (the RedAnt),NaN,NaN,en-gb,0,https://assets.podomatic.net/ts/b8/34/09/steve...,Music,NaN,2016-11-26T18:37:07,2016-11-26T18:37:07
4,0005eac4-fa5d-3c33-a354-ad7928f3705b,http://api.spokenlayer.com/feed/channel/v1-wir...,WIRED News – Spoken Edition,NaN,Get in-depth coverage of current and future tr...,Get in-depth coverage of current and future tr...,WIRED,distribution@spokenlayer.com,http://www.spokenlayer.com,en,0,\n http://media.spokenlayer...,Technology,NaN,2016-11-27T06:46:08,2016-11-25T08:11:13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32827,fffc33ee-0baa-3f4d-82c3-f71bbda270d5,http://www.blogtalkradio.com/special-needs-kid...,Special Needs Kids,"Live interviews with Authors, Parents, caregiv...","Live interviews with Authors, Parents, caregiv...",NaN,Special Needs Kids,feeds@blogtalkradio.com,NaN,en,0,http://www.blogtalkradio.com/api/image/resize/...,Family,Kids & Family,2016-11-27T04:59:03,2016-11-27T04:59:03
32828,fffec846-eced-3c42-b2a1-e91ff963fea5,http://itsrainmakingtime.com/feed/podcast/,It's Rainmaking Time!®,It's Rainmaking Time!®,"Discovery, New & Ancient Knowledge, Imaginatio...",It's Rainmaking Time!® showcases breakthroughs...,Kim Greenhouse,timing@rainmaking.com,https://itsrainmakingtime.com,en-us,0,https://itsrainmakingtime.com/_theme/kahuna-bl...,Business,Religion & Spirituality,2016-11-26T19:19:53,2016-11-19T17:59:58
32829,fffefb5c-ce62-3f0f-bbe3-4ba235196282,http://poochcafe.wm.wizzard.tv/rss,Pooch Café Animated Cartoons,NaN,"Poncho had a perfect little dog thing going, u...","Poncho had a perfect little dog thing going, u...",RingTales,info@ringtales.com,http://www.ringtales.com,en,0,http://static.libsyn.com/p/assets/3/2/d/0/32d0...,Comedy,NaN,2016-11-26T20:15:27,2016-11-07T14:05:00
32830,ffff34aa-8e02-3f42-9ddd-68bd0d22213f,http://feeds.feedburner.com/oitnbpodcast,Orange Is The New Black,OITNB Episode Recaps and Commentary,OITNB Episode Recaps and Commentary,Orange is the New Black Episode Recaps and Com...,DMoney and Xebrielle,NaN,NaN,en,0,http://www.uberwanky.com/oitnbpodcast2.jpg,TV & Film,Health,2016-11-26T20:55:18,2016-07-10T22:40:24


df_data_kmodes.count()
print(df_data_kmodes['language'].value_counts())
print(df_data_kmodes['category'].value_counts())
print(df_data_kmodes['subcategory'].value_counts())

In [130]:
df_new1 = df.drop(['explicit','link','created_at','image','email','feed_url','last_build_date'], axis=1)
df_new = df_new1[0:10000]
df_new.head()

,id,title,subtitle,description,summary,author,language,category,subcategory
0,0002da5f-49a2-31ba-b44c-cdeabdf113cb,Dispatches from CBC Radio,NaN,CBC Radio's Dispatches host Rick MacInnes-Rae ...,CBC Radio's Dispatches host Rick MacInnes-Rae ...,CBC Radio,en-ca,News & Politics,Public Radio
1,00044845-beb8-35f3-99e2-01dbcfb6eb63,Motivation | Inspiration| Success with Entrepr...,Motivation - Inspiration - Success - Mindset -...,Sam Crowley delivers million dollar motivation...,Sam Crowley delivers million dollar motivation...,Sam Crowley,en-us,Business,Business
2,00048bfa-8363-3f10-ac8e-8f27a31fe11f,Disability411 Podcast,NaN,Disability411 - The Podcast for Disability Inf...,NaN,Beth Case,en-us,Government & Organizations,Business
3,0005ab14-ea88-3038-b82c-86366713e96a,Steven Redant's Podcast,"Energy for your work out, your travels and you...",En example of what happened on my dance floor ...,En example of what happened on my dance floor ...,Steven Redant (the RedAnt),en-gb,Music,NaN
4,0005eac4-fa5d-3c33-a354-ad7928f3705b,WIRED News – Spoken Edition,NaN,Get in-depth coverage of current and future tr...,Get in-depth coverage of current and future tr...,WIRED,en,Technology,NaN


In [131]:
df_data = df_new1[['id', 'author', 'language', 'category', 'subcategory']]
df_data

,id,author,language,category,subcategory
0,0002da5f-49a2-31ba-b44c-cdeabdf113cb,CBC Radio,en-ca,News & Politics,Public Radio
1,00044845-beb8-35f3-99e2-01dbcfb6eb63,Sam Crowley,en-us,Business,Business
2,00048bfa-8363-3f10-ac8e-8f27a31fe11f,Beth Case,en-us,Government & Organizations,Business
3,0005ab14-ea88-3038-b82c-86366713e96a,Steven Redant (the RedAnt),en-gb,Music,NaN
4,0005eac4-fa5d-3c33-a354-ad7928f3705b,WIRED,en,Technology,NaN
...,...,...,...,...,...
32827,fffc33ee-0baa-3f4d-82c3-f71bbda270d5,Special Needs Kids,en,Family,Kids & Family
32828,fffec846-eced-3c42-b2a1-e91ff963fea5,Kim Greenhouse,en-us,Business,Religion & Spirituality
32829,fffefb5c-ce62-3f0f-bbe3-4ba235196282,RingTales,en,Comedy,NaN
32830,ffff34aa-8e02-3f42-9ddd-68bd0d22213f,DMoney and Xebrielle,en,TV & Film,Health


In [132]:
from kmodes.kmodes import KModes
df_data_kmodes = df_data[['language','category','subcategory']]
df_data_kmodes = df_data_kmodes.set_index(df_data['id'])
kmode = KModes(n_clusters=3, init = "random", n_init = 5, verbose=1)
df_data_kmodes.isnull().sum()

language         353
category        2423
subcategory    13359
dtype: int64

In [133]:
df_data_kmodes['language'] = df_data_kmodes['language'].fillna('No language')
df_data_kmodes['category'] = df_data_kmodes['category'].fillna('No category')
df_data_kmodes['subcategory'] = df_data_kmodes['subcategory'].fillna('No subcategory')
df_data_kmodes

,language,category,subcategory
id,,,
0002da5f-49a2-31ba-b44c-cdeabdf113cb,en-ca,News & Politics,Public Radio
00044845-beb8-35f3-99e2-01dbcfb6eb63,en-us,Business,Business
00048bfa-8363-3f10-ac8e-8f27a31fe11f,en-us,Government & Organizations,Business
0005ab14-ea88-3038-b82c-86366713e96a,en-gb,Music,No subcategory
0005eac4-fa5d-3c33-a354-ad7928f3705b,en,Technology,No subcategory
...,...,...,...
fffc33ee-0baa-3f4d-82c3-f71bbda270d5,en,Family,Kids & Family
fffec846-eced-3c42-b2a1-e91ff963fea5,en-us,Business,Religion & Spirituality
fffefb5c-ce62-3f0f-bbe3-4ba235196282,en,Comedy,No subcategory


In [134]:
kmode = KModes(n_clusters=50, init = "random", n_init = 5, verbose=1)
clusters = kmode.fit_predict(df_data_kmodes)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 3148, cost: 25047.0
Run 1, iteration: 2/100, moves: 923, cost: 24831.0
Run 1, iteration: 3/100, moves: 22, cost: 24831.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 2, iteration: 1/100, moves: 5884, cost: 26111.0
Run 2, iteration: 2/100, moves: 58, cost: 26111.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 3, iteration: 1/100, moves: 4625, cost: 26381.0
Run 3, iteration: 2/100, moves: 1355, cost: 26234.0
Run 3, iteration: 3/100, moves: 100, cost: 26234.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 4, iteration: 1/100, moves: 5198, cost: 23813.0
Run 4, iteration: 2/100, moves: 810, cost: 23813.0
Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 5, iteration: 1/100, moves: 4284, cost: 26243.0
Run 5, iteration: 2/100, moves: 102

In [135]:
df_data_kmodes.insert(0, "Cluster", clusters, True)

In [136]:
df_data_kmodes.head(20)

,Cluster,language,category,subcategory
id,,,,
0002da5f-49a2-31ba-b44c-cdeabdf113cb,20,en-ca,News & Politics,Public Radio
00044845-beb8-35f3-99e2-01dbcfb6eb63,30,en-us,Business,Business
00048bfa-8363-3f10-ac8e-8f27a31fe11f,30,en-us,Government & Organizations,Business
0005ab14-ea88-3038-b82c-86366713e96a,10,en-gb,Music,No subcategory
0005eac4-fa5d-3c33-a354-ad7928f3705b,2,en,Technology,No subcategory
00060a91-99aa-35fe-b26d-d7977f924a49,19,fr,Education,No subcategory
00083afd-067c-3be9-886f-f95c580e57a9,1,en-us,No category,No subcategory
000a1a7e-755d-3aca-a49d-c1bba00ec26b,36,en,News & Politics,No subcategory
000cdf71-6757-36a9-8aeb-479ea985b3f0,7,en-us,Technology,No subcategory


In [138]:
# Cluster centorid
kmode.cluster_centroids_
# Check the iteration of the clusters created
kmode.n_iter_
# Check the cost of the clusters created
kmode.cost_

23813.0

In [180]:
df_ex_preidct = pd.DataFrame()
df_ex_preidct['language'] = ['en-us', 'en-us', 'en']
df_ex_preidct['category'] = ['Health', 'News & Politics', 'Music']
df_ex_preidct['subcategory'] = ['No subcategory', 'Education', 'Music']
df_ex_preidct['id'] = ['1','2','3']
df_ex_preidct = df_ex_preidct.set_index('id')
df_ex_preidct

,language,category,subcategory
id,,,
1,en-us,Health,No subcategory
2,en-us,News & Politics,Education
3,en,Music,Music


In [188]:
predict_array = kmode.predict(df_ex_preidct)
predict_array
# 랜덤 행의 값을 찾아서 kmode 형식으로 진행을 마저 진행을 해본다.

array([46,  9,  5], dtype=uint16)

In [191]:
if predict_array[0] == 9:
    print('yes')
else:
    print('No')

No


In [230]:
df_data_kmodes_cp = df_data_kmodes.copy()
forex = df_data_kmodes['Cluster'] == predict_array[2]
df_data_kmodes_cp[forex].sample(n=5)


,Cluster,language,category,subcategory
id,,,,
3bae3508-f603-3a35-aafc-5cd8822027a7,5,en-gb,Music,Music
e1214d37-f104-3a29-925a-ecfc6c647c5d,5,es,Music,Music
4c2878d0-a53f-3f29-995b-9cdc6b8fb7b3,5,en-us,Music,Music
567cb7a6-cc4c-34a2-b730-a3bf53bb0ecb,5,en,Music,Music
d36fcb8d-c1a3-3ba7-81c2-e33150679117,5,en-uk,Music,Politics


In [231]:
df_data_kmodes_cp = df_data_kmodes.copy()
forex = df_data_kmodes['Cluster'] == predict_array[1]
df_data_kmodes_cp[forex].sample(n=5)

,Cluster,language,category,subcategory
id,,,,
189547a1-1806-373f-b6ba-b406c6d7e4ac,9,en-us,Education,TV & Film
033eaea8-15e1-3912-b28c-151ba2ff3172,9,en-us,Business,Education
2cb299fb-b71b-3b80-b63f-f8ec64d00636,9,es,Education,Education
4f737f45-0377-3999-ae2a-5b0c075dbbaa,9,en-us,Education,Education
f6f6c067-f985-3e7e-9fd0-9e2cd41453bb,9,en-us,Business,Education
